<a href="https://colab.research.google.com/github/Ninevv/RAG_Project/blob/main/Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dependencies#

In [ ]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install torch
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

In [3]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


#First tests#

In [4]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

In [5]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [6]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [7]:
user_prompt = "What is a leisure activity that you like?"

In [8]:
user_input = "I like to hike"

In [9]:
return_response(user_input, corpus_of_documents)

'Go for a hike and admire the natural scenery.'

#More dependencies#

In [ ]:
!pip install huggingface_hub

In [11]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HF_TOKEN"

In [12]:
from langchain_community.llms import HuggingFaceEndpoint

In [13]:
import torch

In [14]:
# If you want more control, you will need to define the tokenizer and model.
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

In [ ]:
!pip install bitsandbytes==0.43.0
!pip install accelerate

In [ ]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# With pipeline, just specify the task and the model id from the Hub.
from transformers import pipeline
pipe = pipeline("text-generation", model=model,tokenizer=tokenizer, max_new_tokens=10)

In [ ]:
pipe("What is the name of this restaurant?")

#Indexing : Load#

In [18]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [75]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("RAG_Data/Definition of literary movement.pdf", extract_images=True)
pages = loader.load_and_split()

In [76]:
len(pages[0].page_content)

2401

In [77]:
print(pages[0].page_content[:500])

Definition of literary movement  
  
A literary movement is a term that brings together authors whose writing style and vision of 
literature share many common traits. Sometimes, these authors form veritable schools (such as 
naturalism) with a common project.  
  
   
  
Humanism  
  
Humanism, a 19th -century term, refers to a European literary and philosophical cultural movement 
of the 15th and 16th centuries, corresponding to the Renaissance.  
  
In France, the main figures of humanism as 


#Indexing : Split#

In [78]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)

In [79]:
len(all_splits)

61

In [80]:
len(all_splits[0].page_content)

954

In [81]:
all_splits[10].metadata

{'source': 'RAG_Data/Definition of literary movement.pdf',
 'page': 3,
 'start_index': 0}

#Context aware splitter rag q&a quickstart#

#Indexing : Store#

In [82]:
from langchain_community.vectorstores import Chroma #vs FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# create the open-source embedding function
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding_function, persist_directory="./chroma_db")

#Retrieval and Generation : Retrieve #

In [83]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
len(retrieved_docs)

6

In [84]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


In [85]:
# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = vectorstore.similarity_search(query)

# print results
print(docs[0].page_content)

You should only respond in JSON format as described below
Response Format:
{
    "thoughts": {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to user"
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
}
Ensure the response can be parsed by Python json.loads
GPT-Engineer is another project to create a whole repository of code given a task specified in natural language. The GPT-Engineer is instructed to think over a list of smaller components to build and ask for user input to clarify questions as needed.
Here are a sample conversation for task clarification sent to OpenAI ChatCompletion endpoint used by GPT-Engineer. The user inputs are wrapped in {{user input text}}.
[
  {
    "role": "system",


#Retrieval and Generation : Generate #

In [86]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

In [87]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [89]:
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("what is a literary movement?")

'A literary movement is a term that brings together authors whose writing style and vision of literature share many common traits. It can sometimes involve authors forming schools with a common project.'

#Optional : costumize prompt#

In [ ]:
#costumize prompt
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

#Optional#

In [ ]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)